In [1]:
import pandas as pd
import requests
import numpy as np
!conda install -c conda-forge beautifulsoup4 --yes
!conda install -c conda-forge lxml --yes
from bs4 import BeautifulSoup

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.



In [2]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"

In [3]:
response = requests.get(url)
soup = BeautifulSoup(response.text,'lxml')
table = soup.find_all('table')[0] 
column_headers = [th.getText() for th in 
                  soup.findAll('tr', limit=1)[0].findAll('th')]
column_headers[-1] = column_headers[-1].strip()
data_rows = table.findAll('tr')[1:]
row_data = [[td.getText() for td in data_rows[i].findAll('td')]
            for i in range(len(data_rows))]
df = pd.DataFrame(row_data, columns=column_headers)
df.Neighbourhood = df.Neighbourhood.map(lambda x: str(x)[:-1])

In [4]:
df1 = df[df.Borough != 'Not assigned']
df1['Neighbourhood'] = np.where(df1.Neighbourhood == 'Not assigned', df1.Borough, df1.Neighbourhood)
df1.reset_index(inplace=True)
df1 = df1.drop(['index'], axis=1)

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [5]:
df2 = df1.groupby(['Postcode', 'Borough'], sort=False)['Neighbourhood'].apply(lambda x: ', '.join(x))
df3 = pd.DataFrame(df2).reset_index(inplace=False)
df3[0:5]

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [6]:
df3.shape

(103, 3)